In [1]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
f=open('/kaggle/input/chatbot-description/chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()# converts to lowercase

In [4]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences
word_tokens = nltk.word_tokenize(raw)# converts to list of words

lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [5]:
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

WEATHER_RESPONSES = [
    "It's a beautiful day outside!",
    "It's a bit chilly today, but it's supposed to warm up later.",
    "It's raining today, so be sure to bring an umbrella!",
    "The weather forecast shows sunshine throughout the day.",
    "Expect a high temperature today. Stay hydrated!",
    "It's a perfect day for outdoor activities.",
]

JOKES = [
    "What do you call a fish with no eyes? Fsh!",
    "What do you call a cow with no legs? Ground beef!",
    "Why did the scarecrow win an award? Because he was outstanding in his field!",
    "What do you call a lazy kangaroo? A pouch potato!",
    "What do you call a deer with no eyes? No idear!",
    "Why don't scientists trust atoms? Because they make up everything!",
    "Why don't skeletons fight each other? They don't have the guts!",
]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

def jokes():
    return random.choice(JOKES)

def name(sentence):
    name = sentence.split()[1]
    return f"Hello {name}! How can I help you today?"

def weather():
    return random.choice(WEATHER_RESPONSES)

def response(user_response):
    robo_response = ''
    sent_tokens.append(user_response)

    # Create a TF-IDF vectorizer and transform the chat history
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)

    # Calculate the cosine similarity between the user's response and the chat history
    vals = cosine_similarity(tfidf[-1], tfidf)

    # Find the most similar response in the chat history
    idx = vals.argsort()[0][-2]

    # Get the TF-IDF value of the most similar response
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    # If the TF-IDF value of the most similar response is low, then return a default response
    if req_tfidf == 0:
        robo_response += "I am sorry! I don't understand you."
    else:
        robo_response += sent_tokens[idx]

    return robo_response

In [6]:
flag = True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")

while flag:
    user_response = input("Human: ").lower()

    if user_response == "bye":
        flag = False
        print("ROBO: Bye! Take care.")
    elif user_response == "thanks" or user_response == "thank you":
        flag = False
        print("ROBO: You're welcome!")
    else:
        greeting_response = greeting(user_response)
        joke_response = jokes()

        if greeting_response is not None:
            print(greeting_response)
        elif user_response.startswith("tell me a joke"):
            print(joke_response)
        elif user_response.startswith("what's your name"):
            print("ROBO: My name is Robo. How can I assist you?")
        elif "name" in user_response:
            print("ROBO: My name is Robo. How can I assist you?")
        elif "how are you" in user_response:
            print("ROBO: I'm a bot, so I don't have feelings, but thanks for asking!")
        elif user_response.startswith("what's the weather like"):
            print(weather())
        else:
            print(response(user_response))

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!


StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.